In [2]:
!pip install xgboost
#15)
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
df = pd.read_csv("/content/engagement_prediction_dataset.csv")
df["word_count"] = df["post_content"].apply(lambda x: len(str(x).split()))
df["hashtag_count"] = df["hashtags_used"].apply(lambda x: len(re.findall(r"#\w+", str(x))))
df["engagement_rate"] = (df["likes"] + df["shares"] + df["comments"]) / df["word_count"]

# Convert time to numerical features (hour)
df["hour_posted"] = df["time_posted"].apply(lambda x: int(str(x).split(":")[0]))

# Selecting Features
X = df[["word_count", "hashtag_count", "hour_posted"]]
y = df["engagement_rate"]

# Train-test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train Model (Using XGBoost)
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate Model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")

# Save predictions to a new CSV file
# Convert df_test to DataFrame with y_test's index to align indices
df_test = pd.DataFrame(X_test, columns=["word_count", "hashtag_count", "hour_posted"], index=y_test.index)
df_test["actual_engagement_rate"] = y_test.values
df_test["predicted_engagement_rate"] = y_pred
df_test.to_csv("engagement_prediction_results.csv", index=False)

Mean Squared Error: 6811.06
R² Score: nan


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
